This tutorial illustrates a few of the visualization capabilities from `utils/visualizations.py`.

The `visualizations.py` file contains several plotting functionalities. We'll use some existing data to demonstrate their functionality.

**Note:** In this case, the example data is filtered to be seconds apart, in the regular
setting, such measurements would be removed. To prevent this from happening,
we set remove_timing_outliers to False here. For the full dataset, set this flag to True

In [ ]:
import gnss_lib_py as glp

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = glp.AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Since `NavData` is simply a data structure, you can still pull values from it and use standard Python plotting tools like `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(derived_data["gps_millis"],derived_data["raw_pr_m"])
plt.xlabel("GPS Milliseconds")
plt.ylabel("Raw Pseudorange [m]")
plt.title("Matplotlib Plotting Example")
plt.show()

# Visualizations

`gnss_lib_py` extends `matplotlib` and `plotly` functionaltiy to allow quick visuialization of data rows through simple to use function calls.

## Plot Metric

`plot_metric` is the most basic plotting function to plot a row. You need to input the `navdata` object and what row(s) you want to plot. For single rows, it will plot against a default index.

In [ ]:
glonass_data = derived_data.where("gnss_id","glonass")
fig = glp.plot_metric(glonass_data, "raw_pr_m", linestyle="None")

You can also use `plot_metric` to plot two rows against each other.

In [ ]:
fig = glp.plot_metric(glonass_data, "iono_delay_m", "tropo_delay_m", linestyle="none")

For a more informative plot, you can also use the `groupby` flag to choose a row by which to group.

In [ ]:
fig = glp.plot_metric(glonass_data, "iono_delay_m", "tropo_delay_m", groupby="sv_id")

`plot_metric` also allows you to pass in arbitrary arguments to the `matplotlib.pyplot.plot` function call as additional arguments like `linestyle="None"` to remove lines, `marker="None"` to remove the markers, etc.

In [ ]:
fig = glp.plot_metric(glonass_data, "iono_delay_m", "tropo_delay_m", groupby="sv_id",
                  linestyle="None", markeredgecolor="g", markersize=12, 
                  markeredgewidth=1.0)

By default, the visualization tools do not save to file, but you can save them to file by setting the `save` flag to `True`. Check the [reference documentation](file:///home/derek/gnss_lib_py/docs/build/html/reference/utils/visualizations.html#visualizations.plot_metric) for full explanations. Setting `save=True` will default to saving in the `Results` folder, but exact locations can be specified with `fname`. 

## Plot Metric by Constellation

`plot_metric_by_constellation` is similar to `plot_metric` but it breaks up the data into separate figures by constellation and signal type. You need values for rows called `sv_id` and `signal_type` to use this function. We can quickly see that the raw pseudorange value is much more informative when we can separate by constellation and satellite!

In [ ]:
galileo_data = derived_data.where("gnss_id","galileo")
fig = glp.plot_metric_by_constellation(galileo_data, "gps_millis", "raw_pr_m")

## Plot Skyplot

The `plot_skyplot` function plots the satellite skyplot using the satellite positions and estimate receiver position.

In [ ]:
state_estimate = glp.solve_wls(derived_data)

fig = glp.plot_skyplot(derived_data, state_estimate)

## Plot Map

The `plot_map` function allows you to plot latitude and longitude rows of data on a map. The rows must match the standard naming style of `lat_*_deg` and `lon_*_deg` where `*` can be replaced with an arbitrary string.

In [ ]:
fig = glp.plot_map(state_estimate)
fig.show()

You can plot multiple data traces on the same graph as long as the `*` in the `lat_*_deg` and `lon_*_deg` fields is different for each data trace.

In [ ]:
# load a separate data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4_ground_truth.csv --quiet -O "Pixel4_truth.csv"
truth_data_second_trace = glp.AndroidGroundTruth2021("Pixel4_truth.csv")

fig = glp.plot_map(state_estimate, truth_data_second_trace)
fig.show()